In [133]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Nathan
[nltk_data]     Kinalendele\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [134]:
import numpy as np
import json
import glob

import re
import string

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim
from sklearn.manifold import MDS


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [135]:
def preprocess_text(text):
    # Remove code snippets enclosed in <code> tags
    text = re.sub(r'<code>(.*?)</code>', '', text, flags=re.DOTALL)
    
    # Remove HTML tags
    text = re.sub('<[^<]+?>', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove leading/trailing white spaces
    text = text.strip()
    
    return text

# def load_data(file):
#     with open(file, "r", encoding="utf-8") as f:
#         data = json.load(f)
#         texts = []
#         for item in data:
#             title = item["title"]
#             body = item["body"]
#             question_text = title + " " + body
#             texts.append(question_text)
#         return texts
    
    
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        texts = []
        for item in data:
            title = item["title"]
            body = item["body"]
            question_text = title + " " + body
            preprocessed_text = preprocess_text(question_text)
            texts.append(preprocessed_text)
        return texts
    
def write_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
        
file_path = "ne.json"
corpus = load_data(file_path)
write_data("preprocessed_data.json", corpus)     

In [136]:
data = load_data("ne.json")

print (data[0])

what are the differences between github actions and other ci tools like jenkins github announced an upcoming feature github actions

im positive on the benefits of ci tools like jenkins for automatic building or testing which github actions is aimed to be used for in the future

having a repository on github and using an external ci tool has the huge benefit of allowing to move the repository to another git repository platform or even local without rewriting of the whole ci process with github actions youre more or less tied to the github ecosystem

i assume the integration of githubs actions will be more fluent in the native environment but are there any other advantages or disadvantages besides that


In [137]:

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
                
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0])

difference action other ci tool announce upcoming feature action m positive benefit tool jenkin automatic building testing action aim use future have repository use external ci tool huge benefit allow move repository git repository platform even local rewrite whole ci process action re more less tie assume integration github action more fluent native environment other advantage disadvantage


In [138]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess( text, deacc=True)
        final.append(new)
        
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0])

['difference', 'action', 'other', 'ci', 'tool', 'announce', 'upcoming', 'feature', 'action', 'positive', 'benefit', 'tool', 'jenkin', 'automatic', 'building', 'testing', 'action', 'aim', 'use', 'future', 'have', 'repository', 'use', 'external', 'ci', 'tool', 'huge', 'benefit', 'allow', 'move', 'repository', 'git', 'repository', 'platform', 'even', 'local', 'rewrite', 'whole', 'ci', 'process', 'action', 're', 'more', 'less', 'tie', 'assume', 'integration', 'github', 'action', 'more', 'fluent', 'native', 'environment', 'other', 'advantage', 'disadvantage']


In [139]:
bigrams_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigram(data_bigrams)

print(data_bigrams_trigrams[0])

['difference', 'action', 'other', 'ci', 'tool', 'announce', 'upcoming', 'feature', 'action', 'positive', 'benefit', 'tool', 'jenkin', 'automatic', 'building', 'testing', 'action', 'aim', 'use', 'future', 'have', 'repository', 'use', 'external', 'ci', 'tool', 'huge', 'benefit', 'allow', 'move', 'repository', 'git', 'repository', 'platform', 'even', 'local', 'rewrite', 'whole', 'ci', 'process', 'action', 're', 'more', 'less', 'tie', 'assume', 'integration', 'github', 'action', 'more', 'fluent', 'native', 'environment', 'other', 'advantage', 'disadvantage']


In [140]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print (corpus[0])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow
    

[(0, 5), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 3), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 1), (37, 1), (38, 3), (39, 1), (40, 2), (41, 1)]


In [141]:
import json

def get_corpus_size(json_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    num_documents = len(data)
    total_tokens = 0

    for post in data:
        # Assuming the "body" and "title" fields contain the text of the document
        doc_text = post['body'] + " " + post['title']
        words = doc_text.split()  # Split the document into words/tokens
        total_tokens += len(words)

    return num_documents, total_tokens

# Replace 'your_json_file.json' with the path to your JSON file
json_file_path = 'ne.json'
num_documents, total_tokens = get_corpus_size(json_file_path)

print(f"Number of documents in the corpus: {num_documents}")
print(f"Total number of tokens in the corpus: {total_tokens}")


Number of documents in the corpus: 5717
Total number of tokens in the corpus: 1097457


In [26]:
import time
import gensim
from gensim.models import LdaModel
from gensim import corpora

# Preprocessed texts
texts = data_bigrams_trigrams  # Your preprocessed texts here

# Create a dictionary from the texts
dictionary = corpora.Dictionary(data_bigrams_trigrams)

# Create a corpus (bag of words representation)
corpus = [dictionary.doc2bow(text) for text in texts]

# Define the ranges for the parameters
num_topics_range = range(5, 15)  # Number of topics: 2 to 50
num_iterations_range = range(100, 500, 100)  # Number of iterations: 10 to 500 (step size: 10)
chunk_size_range = range(1000, 2000, 100)  # Chunk size: 10 to 200 (step size: 10)
passes_range = range(10, 21)  # Number of passes: 1 to 100

# Initialize variables for tracking the optimal parameters and coherence score
optimal_params = {}
max_coherence_score = 0

# Start measuring the overall execution time
start_time = time.time()

# Iterate over the parameter ranges and train LDA models
for num_topics in num_topics_range:
    for num_iterations in num_iterations_range:
        for chunk_size in chunk_size_range:
            for passes in passes_range:
                # Start measuring the execution time for each LDA model
                model_start_time = time.time()

                lda_model = LdaModel(corpus=corpus,
                                     id2word=dictionary,
                                     num_topics=num_topics,
                                     iterations=num_iterations,
                                     chunksize=chunk_size,
                                     random_state=100)
                # Calculate coherence score
                coherence_model = gensim.models.CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
                coherence_score = coherence_model.get_coherence()

                # Check if current coherence score is higher than previous max
                if coherence_score > max_coherence_score:
                    max_coherence_score = coherence_score
                    optimal_params = {
                        'num_topics': num_topics,
                        'num_iterations': num_iterations,
                        'chunk_size': chunk_size,
                        'passes': passes
                    }

                # Calculate the elapsed time for the current LDA model
                model_elapsed_time = time.time() - model_start_time
                print("Elapsed Time for Model (Topics: {}, Iterations: {}, Chunk Size: {}, Passes: {}): {:.2f} seconds, Coherence Score: {:.4f}".format(num_topics, num_iterations, chunk_size, passes, model_elapsed_time, coherence_score))

# Calculate the overall elapsed time
elapsed_time = time.time() - start_time

# Print the optimal parameters and coherence score
print("Optimal Parameters:")
print("Number of Topics:", optimal_params['num_topics'])
print("Number of Iterations:", optimal_params['num_iterations'])
print("Chunk Size:", optimal_params['chunk_size'])
print("Passes:", optimal_params['passes'])
print("Coherence Score:", max_coherence_score)
print("Overall Execution Time: {:.2f} seconds".format(elapsed_time))


Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 10): 14.81 seconds, Coherence Score: 0.3500
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 11): 11.85 seconds, Coherence Score: 0.3500
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 12): 13.39 seconds, Coherence Score: 0.3500
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 13): 12.38 seconds, Coherence Score: 0.3500
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 14): 11.31 seconds, Coherence Score: 0.3500
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 15): 11.12 seconds, Coherence Score: 0.3500
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 16): 11.15 seconds, Coherence Score: 0.3500
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1000, Passes: 17): 11.21 seconds, Coherence Score: 0.3500
Elapsed Time for Model (

Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 12): 15.30 seconds, Coherence Score: 0.3511
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 13): 14.98 seconds, Coherence Score: 0.3511
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 14): 14.29 seconds, Coherence Score: 0.3511
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 15): 19.01 seconds, Coherence Score: 0.3511
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 16): 12.43 seconds, Coherence Score: 0.3511
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 17): 30.11 seconds, Coherence Score: 0.3511
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 18): 18.70 seconds, Coherence Score: 0.3511
Elapsed Time for Model (Topics: 5, Iterations: 100, Chunk Size: 1600, Passes: 19): 21.83 seconds, Coherence Score: 0.3511
Elapsed Time for Model (

Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1200, Passes: 14): 15.95 seconds, Coherence Score: 0.3308
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1200, Passes: 15): 14.45 seconds, Coherence Score: 0.3308
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1200, Passes: 16): 16.59 seconds, Coherence Score: 0.3308
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1200, Passes: 17): 19.86 seconds, Coherence Score: 0.3308
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1200, Passes: 18): 20.04 seconds, Coherence Score: 0.3308
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1200, Passes: 19): 16.87 seconds, Coherence Score: 0.3308
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1200, Passes: 20): 13.45 seconds, Coherence Score: 0.3308
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1300, Passes: 10): 15.29 seconds, Coherence Score: 0.3417
Elapsed Time for Model (

Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1800, Passes: 16): 14.70 seconds, Coherence Score: 0.3333
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1800, Passes: 17): 14.83 seconds, Coherence Score: 0.3333
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1800, Passes: 18): 17.79 seconds, Coherence Score: 0.3333
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1800, Passes: 19): 14.14 seconds, Coherence Score: 0.3333
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1800, Passes: 20): 15.40 seconds, Coherence Score: 0.3333
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1900, Passes: 10): 15.40 seconds, Coherence Score: 0.3393
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1900, Passes: 11): 14.27 seconds, Coherence Score: 0.3393
Elapsed Time for Model (Topics: 5, Iterations: 200, Chunk Size: 1900, Passes: 12): 20.35 seconds, Coherence Score: 0.3393
Elapsed Time for Model (

Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1400, Passes: 18): 16.38 seconds, Coherence Score: 0.3374
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1400, Passes: 19): 13.84 seconds, Coherence Score: 0.3374
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1400, Passes: 20): 15.33 seconds, Coherence Score: 0.3374
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1500, Passes: 10): 14.87 seconds, Coherence Score: 0.3440
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1500, Passes: 11): 16.85 seconds, Coherence Score: 0.3440
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1500, Passes: 12): 17.38 seconds, Coherence Score: 0.3440
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1500, Passes: 13): 22.93 seconds, Coherence Score: 0.3440
Elapsed Time for Model (Topics: 5, Iterations: 300, Chunk Size: 1500, Passes: 14): 20.14 seconds, Coherence Score: 0.3440
Elapsed Time for Model (

...

Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1400, Passes: 15): 11.69 seconds, Coherence Score: 0.3194
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1400, Passes: 16): 11.96 seconds, Coherence Score: 0.3194
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1400, Passes: 17): 11.97 seconds, Coherence Score: 0.3194
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1400, Passes: 18): 11.89 seconds, Coherence Score: 0.3194
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1400, Passes: 19): 11.95 seconds, Coherence Score: 0.3194
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1400, Passes: 20): 11.92 seconds, Coherence Score: 0.3194
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1500, Passes: 10): 12.86 seconds, Coherence Score: 0.3206
Elapsed Time for Model (Topics: 14, Iterations: 200, Chunk Size: 1500, Passes: 11): 12.78 seconds, Coherence Score: 0.3206
Elapsed Time for

Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1000, Passes: 16): 12.20 seconds, Coherence Score: 0.3297
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1000, Passes: 17): 11.92 seconds, Coherence Score: 0.3297
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1000, Passes: 18): 12.27 seconds, Coherence Score: 0.3297
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1000, Passes: 19): 11.84 seconds, Coherence Score: 0.3297
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1000, Passes: 20): 12.29 seconds, Coherence Score: 0.3297
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1100, Passes: 10): 11.84 seconds, Coherence Score: 0.3288
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1100, Passes: 11): 11.97 seconds, Coherence Score: 0.3288
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1100, Passes: 12): 12.03 seconds, Coherence Score: 0.3288
Elapsed Time for

Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1600, Passes: 17): 12.61 seconds, Coherence Score: 0.3215
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1600, Passes: 18): 13.38 seconds, Coherence Score: 0.3215
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1600, Passes: 19): 12.64 seconds, Coherence Score: 0.3215
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1600, Passes: 20): 13.31 seconds, Coherence Score: 0.3215
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1700, Passes: 10): 12.67 seconds, Coherence Score: 0.3250
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1700, Passes: 11): 13.23 seconds, Coherence Score: 0.3250
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1700, Passes: 12): 12.73 seconds, Coherence Score: 0.3250
Elapsed Time for Model (Topics: 14, Iterations: 300, Chunk Size: 1700, Passes: 13): 12.61 seconds, Coherence Score: 0.3250
Elapsed Time for

Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1200, Passes: 18): 12.41 seconds, Coherence Score: 0.3149
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1200, Passes: 19): 12.90 seconds, Coherence Score: 0.3149
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1200, Passes: 20): 12.60 seconds, Coherence Score: 0.3149
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1300, Passes: 10): 12.35 seconds, Coherence Score: 0.3135
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1300, Passes: 11): 12.17 seconds, Coherence Score: 0.3135
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1300, Passes: 12): 12.31 seconds, Coherence Score: 0.3135
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1300, Passes: 13): 12.77 seconds, Coherence Score: 0.3135
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1300, Passes: 14): 12.49 seconds, Coherence Score: 0.3135
Elapsed Time for

Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1800, Passes: 19): 12.75 seconds, Coherence Score: 0.3211
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1800, Passes: 20): 12.96 seconds, Coherence Score: 0.3211
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1900, Passes: 10): 12.56 seconds, Coherence Score: 0.2972
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1900, Passes: 11): 12.55 seconds, Coherence Score: 0.2972
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1900, Passes: 12): 12.94 seconds, Coherence Score: 0.2972
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1900, Passes: 13): 12.66 seconds, Coherence Score: 0.2972
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1900, Passes: 14): 12.76 seconds, Coherence Score: 0.2972
Elapsed Time for Model (Topics: 14, Iterations: 400, Chunk Size: 1900, Passes: 15): 12.66 seconds, Coherence Score: 0.2972
Elapsed Time for

In [142]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 

                                            id2word=id2word, 
                                            num_topics = 12,
passes = 200,
chunksize = 1200,
iterations=100,
alpha = 'auto',
eta = 'auto',
random_state = 100


                                            ) 


In [151]:
import gensim

# Assuming you have already trained your LDA model and have it in the variable lda_model
# Assuming you have the corpus used for training in the variable corpus

# Get the overall topic distribution for the entire corpus
topic_distribution = [lda_model.get_document_topics(doc, minimum_probability=0.0) for doc in corpus]

# Initialize a list to store the count of questions per topic
num_questions_per_topic = [0] * lda_model.num_topics

# Iterate through the corpus and count the number of questions for each topic
for doc_topics in topic_distribution:
    for topic_id, _ in doc_topics:
        num_questions_per_topic[topic_id] += 1

# Calculate the average probability of each topic
num_topics = lda_model.num_topics
overall_probabilities = [0.0] * num_topics
num_docs = len(corpus)

for doc_topics in topic_distribution:
    for topic_id, probability in doc_topics:
        overall_probabilities[topic_id] += probability

overall_probabilities = [prob / num_docs for prob in overall_probabilities]

# Convert probabilities to percentages
overall_probabilities_percentage = [prob * 100 for prob in overall_probabilities]

# Print the overall probability, number of questions, and keywords of each topic in percentage
for topic_num, (probability, keywords) in enumerate(zip(overall_probabilities_percentage, lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=True))):
    num_questions = num_questions_per_topic[topic_num]
    print(f"Topic {topic_num}: Probability: {probability:.2f}%")
    print(f"Number of Questions: {num_questions}")
    print(f"Keywords: {keywords}\n")


Topic 0: Probability: 6.77%
Number of Questions: 5717
Keywords: (0, '0.087*"deploy" + 0.051*"secret" + 0.048*"app" + 0.032*"azure" + 0.029*"deployment" + 0.029*"server" + 0.027*"api" + 0.025*"key" + 0.025*"page" + 0.024*"use"')

Topic 1: Probability: 36.22%
Number of Questions: 5717
Keywords: (1, '0.036*"error" + 0.032*"try" + 0.029*"use" + 0.029*"get" + 0.029*"workflow" + 0.029*"work" + 0.020*"fail" + 0.019*"step" + 0.018*"here" + 0.016*"find"')

Topic 2: Probability: 5.97%
Number of Questions: 5717
Keywords: (2, '0.130*"branch" + 0.076*"workflow" + 0.075*"trigger" + 0.051*"action" + 0.051*"push" + 0.050*"commit" + 0.044*"pull_request" + 0.035*"merge" + 0.033*"pr" + 0.029*"master"')

Topic 3: Probability: 3.49%
Number of Questions: 5717
Keywords: (3, '0.127*"docker" + 0.121*"image" + 0.082*"container" + 0.044*"build" + 0.031*"dockerfile" + 0.024*"connect" + 0.022*"service" + 0.021*"database" + 0.015*"registry" + 0.013*"push"')

Topic 4: Probability: 1.30%
Number of Questions: 5717
Key

In [193]:
import gensim

# Assuming you have already trained your LDA model and have it in the variable lda_model
# Assuming you have the corpus used for training in the variable corpus

# Get the dominant topic for each document in the corpus
dominant_topic_per_document = [sorted(doc_topics, key=lambda x: x[1], reverse=True)[0][0]
                              for doc_topics in topic_distribution]

# Initialize a list to store the count of documents per dominant topic
num_documents_per_topic = [0] * lda_model.num_topics

# Count the number of documents for each dominant topic
for topic_id in dominant_topic_per_document:
    num_documents_per_topic[topic_id] += 1

# Calculate the average probability of each topic using the dominant topics
num_topics = lda_model.num_topics
overall_probabilities = [0.0] * num_topics

for doc_topics in topic_distribution:
    dominant_topic_id, probability = sorted(doc_topics, key=lambda x: x[1], reverse=True)[0]
    overall_probabilities[dominant_topic_id] += probability

overall_probabilities = [prob / num_docs for prob in overall_probabilities]

# Convert probabilities to percentages
overall_probabilities_percentage = [prob * 100 for prob in overall_probabilities]

# Print the overall probability, number of documents, and keywords of each topic in percentage
for topic_num, (probability, keywords) in enumerate(zip(overall_probabilities_percentage, lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=True))):
    num_documents = num_documents_per_topic[topic_num]
    print(f"Topic {topic_num}: Probability: {probability:.2f}%")
    print(f"Number of Documents: {num_documents}")
    print(f"Keywords: {keywords}\n")


Topic 0: Probability: 0.62%
Number of Documents: 104
Keywords: (0, '0.087*"deploy" + 0.051*"secret" + 0.048*"app" + 0.032*"azure" + 0.029*"deployment" + 0.029*"server" + 0.027*"api" + 0.025*"key" + 0.025*"page" + 0.024*"use"')

Topic 1: Probability: 27.47%
Number of Documents: 3497
Keywords: (1, '0.036*"error" + 0.032*"try" + 0.029*"use" + 0.029*"get" + 0.029*"workflow" + 0.029*"work" + 0.020*"fail" + 0.019*"step" + 0.018*"here" + 0.016*"find"')

Topic 2: Probability: 1.61%
Number of Documents: 245
Keywords: (2, '0.130*"branch" + 0.076*"workflow" + 0.075*"trigger" + 0.051*"action" + 0.051*"push" + 0.050*"commit" + 0.044*"pull_request" + 0.035*"merge" + 0.033*"pr" + 0.029*"master"')

Topic 3: Probability: 0.28%
Number of Documents: 51
Keywords: (3, '0.127*"docker" + 0.121*"image" + 0.082*"container" + 0.044*"build" + 0.031*"dockerfile" + 0.024*"connect" + 0.022*"service" + 0.021*"database" + 0.015*"registry" + 0.013*"push"')

Topic 4: Probability: 0.06%
Number of Documents: 9
Keywords: 

In [198]:
import gensim
import json

# Assuming you have already trained your LDA model and have it in the variable lda_model
# Assuming you have the corpus used for training in the variable corpus

# Load data from "ne.json" file
with open('ne.json', 'r', encoding='utf-8')  as file:
    data = json.load(file)

# Extract favorite_count, score, and view_count from the data and store them in separate lists
favorite_counts = [int(item['favorite_count']) if item['favorite_count'] else 0 for item in data]
scores = [float(item['score']) if item['score'] else 0.0 for item in data]
view_counts = [int(item['view_count']) if item['view_count'] else 0 for item in data]
answer_counts = [int(item['answer_count']) if item['answer_count'] else 0 for item in data]
accepted_answer_ids = [item['accepted_answer_id'] for item in data]

# Rest of the code remains the same as before...

# Get the dominant topic for each document in the corpus
dominant_topic_per_document = [sorted(doc_topics, key=lambda x: x[1], reverse=True)[0][0]
                              for doc_topics in topic_distribution]

# Initialize lists to store the count of documents and sum of favorite_count, score, view_count, unanswered_questions, and questions_without_accepted_answer per dominant topic
num_documents_per_topic = [0] * lda_model.num_topics
sum_favorite_count_per_topic = [0] * lda_model.num_topics
sum_score_per_topic = [0.0] * lda_model.num_topics
sum_view_count_per_topic = [0] * lda_model.num_topics
unanswered_questions_per_topic = [0] * lda_model.num_topics
questions_without_accepted_answer_per_topic = [0] * lda_model.num_topics

# Count the number of documents and sum favorite_count, score, view_count, unanswered_questions, and questions_without_accepted_answer for each dominant topic
for doc_id, topic_id in enumerate(dominant_topic_per_document):
    num_documents_per_topic[topic_id] += 1
    sum_favorite_count_per_topic[topic_id] += favorite_counts[doc_id]
    sum_score_per_topic[topic_id] += scores[doc_id]
    sum_view_count_per_topic[topic_id] += view_counts[doc_id]

    # Check if the question has no answers
    if answer_counts[doc_id] == 0:
        unanswered_questions_per_topic[topic_id] += 1

    # Check if the question has no accepted answer
    if not accepted_answer_ids[doc_id]:
        questions_without_accepted_answer_per_topic[topic_id] += 1

# Calculate the average favorite_count, score, and view_count for each topic
average_favorite_count_per_topic = [sum_count / num_documents_per_topic[topic_id] for topic_id, sum_count in enumerate(sum_favorite_count_per_topic)]
average_score_per_topic = [sum_score / num_documents_per_topic[topic_id] for topic_id, sum_score in enumerate(sum_score_per_topic)]
average_view_count_per_topic = [sum_view / num_documents_per_topic[topic_id] for topic_id, sum_view in enumerate(sum_view_count_per_topic)]

# Convert probabilities to percentages
overall_probabilities_percentage = [prob * 100 for prob in overall_probabilities]

# Print the overall probability, number of documents, average favorite_count, average score, average view_count, unanswered questions, and questions without accepted answer of each topic in percentage
for topic_num, (probability, keywords) in enumerate(zip(overall_probabilities_percentage, lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=True))):
    num_documents = num_documents_per_topic[topic_num]
    avg_favorite_count = average_favorite_count_per_topic[topic_num]
    avg_score = average_score_per_topic[topic_num]
    avg_view_count = average_view_count_per_topic[topic_num]
    unanswered_questions = unanswered_questions_per_topic[topic_num]
    questions_without_accepted_answer = questions_without_accepted_answer_per_topic[topic_num]
    print(f"Topic {topic_num}: Probability: {probability:.2f}%")
    print(f"Number of Documents: {num_documents}")
    print(f"Average favorite_count: {avg_favorite_count:.2f}")
    print(f"Average score: {avg_score:.2f}")
    print(f"Average view_count: {avg_view_count:.2f}")
    print(f"Unanswered Questions: {unanswered_questions}")
    print(f"Questions without Accepted Answer: {questions_without_accepted_answer}")
    print(f"Keywords: {keywords}\n")


Topic 0: Probability: 0.62%
Number of Documents: 104
Average favorite_count: 0.38
Average score: 1.65
Average view_count: 849.11
Unanswered Questions: 32
Questions without Accepted Answer: 68
Keywords: (0, '0.087*"deploy" + 0.051*"secret" + 0.048*"app" + 0.032*"azure" + 0.029*"deployment" + 0.029*"server" + 0.027*"api" + 0.025*"key" + 0.025*"page" + 0.024*"use"')

Topic 1: Probability: 27.47%
Number of Documents: 3497
Average favorite_count: 0.46
Average score: 2.50
Average view_count: 1601.05
Unanswered Questions: 974
Questions without Accepted Answer: 2115
Keywords: (1, '0.036*"error" + 0.032*"try" + 0.029*"use" + 0.029*"get" + 0.029*"workflow" + 0.029*"work" + 0.020*"fail" + 0.019*"step" + 0.018*"here" + 0.016*"find"')

Topic 2: Probability: 1.61%
Number of Documents: 245
Average favorite_count: 0.55
Average score: 3.10
Average view_count: 1746.54
Unanswered Questions: 66
Questions without Accepted Answer: 142
Keywords: (2, '0.130*"branch" + 0.076*"workflow" + 0.075*"trigger" + 0.05

In [206]:
import gensim
import json

# Assuming you have already trained your LDA model and have it in the variable lda_model
# Assuming you have the corpus used for training in the variable corpus

# Load data from "ne.json" file
with open('ne.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Print all data for each question
for question_data in data:
    print("Question ID:", question_data.get('question_id', None))
    print("Title:", question_data.get('title', None))
    print("Body:", question_data.get('body', None))
    print("Tags:", question_data.get('tags', None))
    print("Creation Date:", question_data.get('creation_date', None))
    print("Favorite Count:", question_data.get('favorite_count', None))
    print("Score:", question_data.get('score', None))
    print("View Count:", question_data.get('view_count', None))
    print("Answer Count:", question_data.get('answer_count', None))
    print("Accepted Answer ID:", question_data.get('accepted_answer_id', None))
    print("************************************")

# Assuming you have already calculated the dominant topic for each document in the corpus and stored it in the variable dominant_topic_per_document.
# Rest of the code remains the same as before...

# Get the dominant topic for each document in the corpus
dominant_topic_per_document = [sorted(doc_topics, key=lambda x: x[1], reverse=True)[0][0]
                              for doc_topics in topic_distribution]

# Initialize lists to store the count of documents and sum of favorite_count, score, view_count, unanswered_questions, and questions_without_accepted_answer per dominant topic
num_documents_per_topic = [0] * lda_model.num_topics
sum_favorite_count_per_topic = [0] * lda_model.num_topics
sum_score_per_topic = [0.0] * lda_model.num_topics
sum_view_count_per_topic = [0] * lda_model.num_topics
unanswered_questions_per_topic = [0] * lda_model.num_topics
questions_without_accepted_answer_per_topic = [0] * lda_model.num_topics

# Count the number of documents and sum favorite_count, score, view_count, unanswered_questions, and questions_without_accepted_answer for each dominant topic
for doc_id, topic_id in enumerate(dominant_topic_per_document):
    num_documents_per_topic[topic_id] += 1
    sum_favorite_count_per_topic[topic_id] += favorite_counts[doc_id]
    sum_score_per_topic[topic_id] += scores[doc_id]
    sum_view_count_per_topic[topic_id] += view_counts[doc_id]

    # Check if the question has no answers
    if answer_counts[doc_id] == 0:
        unanswered_questions_per_topic[topic_id] += 1

    # Check if the question has no accepted answer
    if not accepted_answer_ids[doc_id]:
        questions_without_accepted_answer_per_topic[topic_id] += 1

# Calculate the average favorite_count, score, and view_count for each topic
average_favorite_count_per_topic = [sum_count / num_documents_per_topic[topic_id] for topic_id, sum_count in enumerate(sum_favorite_count_per_topic)]
average_score_per_topic = [sum_score / num_documents_per_topic[topic_id] for topic_id, sum_score in enumerate(sum_score_per_topic)]
average_view_count_per_topic = [sum_view / num_documents_per_topic[topic_id] for topic_id, sum_view in enumerate(sum_view_count_per_topic)]

# Calculate the probability of each word belonging to each topic
topic_word_probabilities = lda_model.show_topics(num_topics=lda_model.num_topics, num_words=len(lda_model.id2word), formatted=False)

# Initialize list to store the sum of probabilities for each topic
sum_probabilities_per_topic = [0.0] * lda_model.num_topics

# Calculate the sum of probabilities for each topic
for topic_id, word_probs in topic_word_probabilities:
    for word_id, word_prob in word_probs:
        sum_probabilities_per_topic[topic_id] += word_prob

# Calculate the percentage of occurrence of each topic
total_num_documents = len(dominant_topic_per_document)
percentage_occurrence_per_topic = [(num_docs_topic / total_num_documents) * 100 for num_docs_topic in num_documents_per_topic]

# Print the statistics for each individual topic
for topic_num, (probability, keywords) in enumerate(zip(sum_probabilities_per_topic, lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=True))):
    num_documents = num_documents_per_topic[topic_num]
    avg_favorite_count = average_favorite_count_per_topic[topic_num]
    avg_score = average_score_per_topic[topic_num]
    avg_view_count = average_view_count_per_topic[topic_num]
    unanswered_questions = unanswered_questions_per_topic[topic_num]
    questions_without_accepted_answer = questions_without_accepted_answer_per_topic[topic_num]
    percent_occurrence = percentage_occurrence_per_topic[topic_num]
    print(f"Topic {topic_num}: Probability: {probability:.2f}")
    print(f"Number of Documents: {num_documents}")
    print(f"Average favorite_count: {avg_favorite_count:.2f}")
    print(f"Average score: {avg_score:.2f}")
    print(f"Average view_count: {avg_view_count:.2f}")
    print(f"Unanswered Questions: {unanswered_questions}")
    print(f"Questions without Accepted Answer: {questions_without_accepted_answer}")
    print(f"Percentage of Occurrence: {percent_occurrence:.2f}%")
    print(f"Keywords: {keywords}\n")

# Print the statistics for all topics together
print("All Questions Combined:")
print("************************************")
print(f"Total Number of Documents: {total_num_documents}")
print(f"Average favorite_count: {sum(favorite_counts) / total_num_documents:.2f}")
print(f"Average score: {sum(scores) / total_num_documents:.2f}")
print(f"Average view_count: {sum(view_counts) / total_num_documents:.2f}")
print(f"Total Unanswered Questions: {sum(unanswered_questions_per_topic)}")
print(f"Total Questions without Accepted Answer: {sum(questions_without_accepted_answer_per_topic)}")
print("************************************")
print("Overall Word Probabilities:")
for topic_num, (probability, keywords) in enumerate(zip(sum_probabilities_per_topic, lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=True))):
    print(f"Topic {topic_num}: Probability: {probability:.2f}")
    print(f"Keywords: {keywords}\n")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Topic 0: Probability: 1.00
Number of Documents: 104
Average favorite_count: 0.38
Average score: 1.65
Average view_count: 849.11
Unanswered Questions: 32
Questions without Accepted Answer: 68
Percentage of Occurrence: 1.82%
Keywords: (0, '0.087*"deploy" + 0.051*"secret" + 0.048*"app" + 0.032*"azure" + 0.029*"deployment" + 0.029*"server" + 0.027*"api" + 0.025*"key" + 0.025*"page" + 0.024*"use"')

Topic 1: Probability: 1.00
Number of Documents: 3497
Average favorite_count: 0.46
Average score: 2.50
Average view_count: 1601.05
Unanswered Questions: 974
Questions without Accepted Answer: 2115
Percentage of Occurrence: 61.17%
Keywords: (1, '0.036*"error" + 0.032*"try" + 0.029*"use" + 0.029*"get" + 0.029*"workflow" + 0.029*"work" + 0.020*"fail" + 0.019*"step" + 0.018*"here" + 0.016*"find"')

Topic 2: Probability: 1.00
Number of Documents: 245
Average favorite_count: 0.55
Average score: 3.10
Average view_count: 1746.54
Unanswered Questions: 66
Questions without Accepted Answer: 142
Percentage o

In [179]:
import json
from gensim.models import LdaModel
from gensim import corpora

# Step 1: Read data from the JSON file
with open('ne.json', 'r', encoding='utf-8') as json_file:
    data_list = json.load(json_file)

# Step 2: Assuming you have already trained your LDA model 'lda_model' and have your corpus available.
# (Please ensure you have the 'lda_model' and 'corpus' variables from your previous code.)

# Step 3: Associate topics with documents and find the dominant topic for each document
doc_topics = [lda_model.get_document_topics(doc) for doc in corpus]
dominant_topics = [max(doc_topics, key=lambda x: x[1])[0] for doc_topics in doc_topics]

# Step 4: Group data points (documents) based on their associated topics
topics_data = {topic_id: [] for topic_id in range(lda_model.num_topics)}
for doc_id, topic_id in enumerate(dominant_topics):
    topics_data[topic_id].append(data_list[doc_id])

# Step 5: Calculate averages for each topic
average_fav_count_per_topic = {}
average_score_per_topic = {}
average_view_count_per_topic = {}

# Step 6: Calculate the number of questions without an answer and without an accepted answer for each topic
num_questions_without_answer = {topic_id: 0 for topic_id in range(lda_model.num_topics)}
num_questions_without_accepted_answer = {topic_id: 0 for topic_id in range(lda_model.num_topics)}

total_documents = len(data_list)  # Total number of documents in the corpus

for topic_id, documents in topics_data.items():
    total_fav_count = 0
    total_score = 0
    total_view_count = 0
    num_documents = len(documents)

    for document in documents:
        # Assuming 'favorite_count', 'score', 'view_count', 'answer_count', and 'accepted_answer_id'
        # are attributes in each document dictionary
        total_fav_count += int(document.get('favorite_count', 0) or 0)
        total_score += int(document.get('score', 0) or 0)
        total_view_count += int(document.get('view_count', 0) or 0)

        if document.get('answer_count', 0) == 0:
            num_questions_without_answer[topic_id] += 1
        if document.get('accepted_answer_id') is None:
            num_questions_without_accepted_answer[topic_id] += 1

    average_fav_count_per_topic[topic_id] = total_fav_count / num_documents
    average_score_per_topic[topic_id] = total_score / num_documents
    average_view_count_per_topic[topic_id] = total_view_count / num_documents

# Print the keywords and document distribution for each topic
for topic_id in range(lda_model.num_topics):
    print(f"Topic {topic_id}:")
    keywords = lda_model.show_topic(topic_id, topn=10)  # Display top 10 keywords for each topic
    sum_probabilities = sum(weight for _, weight in keywords)
    for word, weight in keywords:
        print(f"{word}: {weight:.4f}")
    print(f"Sum of Probabilities for all keywords in this topic: {sum_probabilities:.4f}")
    num_documents = len(topics_data[topic_id])
    print(f"Number of documents associated with this topic: {num_documents}")
    print(f"Number of questions without an answer in this topic: {num_questions_without_answer[topic_id]}")
    print(f"Number of questions without an accepted answer in this topic: {num_questions_without_accepted_answer[topic_id]}")
    print()

# Step 7: Calculate overall probability of each topic
overall_topic_probability = {topic_id: len(documents) / total_documents * 100 for topic_id, documents in topics_data.items()}

# Step 8: Print the overall probabilities
for topic_id in range(lda_model.num_topics):
    print(f"Topic {topic_id}: Probability = {overall_topic_probability[topic_id]:.2f}%")
    print()

print("Average favorite_count per topic:", average_fav_count_per_topic)
print("Average score per topic:", average_score_per_topic)
print("Average view_count per topic:", average_view_count_per_topic)


Topic 0:
deploy: 0.0865
secret: 0.0508
app: 0.0484
azure: 0.0319
deployment: 0.0287
server: 0.0287
api: 0.0271
key: 0.0251
page: 0.0251
use: 0.0244
Sum of Probabilities for all keywords in this topic: 0.3768
Number of documents associated with this topic: 104
Number of questions without an answer in this topic: 0
Number of questions without an accepted answer in this topic: 68

Topic 1:
error: 0.0365
try: 0.0316
use: 0.0294
get: 0.0291
workflow: 0.0289
work: 0.0286
fail: 0.0204
step: 0.0187
here: 0.0180
find: 0.0163
Sum of Probabilities for all keywords in this topic: 0.2574
Number of documents associated with this topic: 3497
Number of questions without an answer in this topic: 0
Number of questions without an accepted answer in this topic: 2115

Topic 2:
branch: 0.1300
workflow: 0.0761
trigger: 0.0749
action: 0.0515
push: 0.0515
commit: 0.0501
pull_request: 0.0435
merge: 0.0347
pr: 0.0329
master: 0.0290
Sum of Probabilities for all keywords in this topic: 0.5742
Number of documents a

In [11]:
#warnings.filterwarnings('ignore', category=FutureWarning)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

C:\Users\Nathan Kinalendele\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.418527  0.169965       1        1  35.048894
11    -0.244950  0.347106       2        1  26.829528
0      0.331350 -0.259971       3        1   7.153898
5      0.059585  0.438239       4        1   6.525867
2     -0.118391  0.156711       5        1   5.799270
8     -0.408050 -0.174081       6        1   5.086046
3      0.218145  0.013504       7        1   3.726712
10     0.435394  0.002735       8        1   2.726446
7     -0.136125 -0.185016       9        1   2.286737
9      0.116029 -0.385331      10        1   2.181800
6      0.286042  0.265017      11        1   1.343993
4     -0.120502 -0.388878      12        1   1.290808, topic_info=          Term         Freq        Total Category  logprob  loglift
46        file  4282.000000  4282.000000  Default  10.0000  10.0000
91         run  4641.000000  4641.000000  Default   9.0000   9.0000
92        test  2453.000000  2453.000000  Default   8.0000   8.0000
182   workflow  4050.000000  4050.000000  Default   7.0000   7.0000
122     branch  1918.000000  1918.000000  Default   6.0000   6.0000
...        ...          ...          ...      ...      ...      ...
931      query    39.166196    40.042734  Topic12  -4.4288   4.3278
1597     reuse    56.755859    63.514349  Topic12  -4.0579   4.2374
341   resource    64.521136   123.422754  Topic12  -3.9296   3.7013
925       json    57.834514    98.477464  Topic12  -4.0390   3.8177
1317  instance    59.189678   168.753664  Topic12  -4.0159   3.3022

[189 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2561     10  0.973010     accept
42        1  0.151814     access
42        3  0.341581     access
42        8  0.407136     access
42        9  0.100059     access
...     ...       ...        ...
182       5  0.277273   workflow
1198     10  0.986065  workspace
541       1  0.240377        yml
541       6  0.758523        yml
1244     10  0.986185        zip

[256 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 12, 1, 6, 3, 9, 4, 11, 8, 10, 7, 5])